In [18]:
from core.utils import Tibanna
from dcicutils import ff_utils
from core.utils import run_workflow
from datetime import datetime
from core.wfr import *

env = 'data'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key={"default" : tibanna.ff_keys})


In [11]:
import time
from tasks import run_fastqc
from invoke import run

# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately

run_fastqc = raw_input("Do you wanna run fastqc if missing? (fq/none)")

# url for hic exps
fastq_url = "search/?type=FileFastq&quality_metric.uuid=No+value" + \
            "&status=released&status=released%20to%20project&status=uploaded" + \
            "&limit=all" + \
            "&q=date_created%3A%3E%3D2016-09-01"

fastq_files = ff_utils.search_metadata(fastq_url , ff_env = env)


def step_settings(seq, my_organism=""):
    genome = ""
    mapper = {'human':'GRCh38','mouse':'GRCm38'}
    genome = mapper.get(my_organism)
    
    wf_dict =[{
        'wf_name': 'fastqc-0-11-4-1',
        'wf_uuid': '2324ad76-ff37-4157-8bcc-3ce72b7dace9',
        'parameters': {}
        }]
    return wf_dict[seq]

raw_bucket = tibanna.s3.raw_file_bucket
out_bucket = tibanna.s3.outfile_bucket

counter = 0
completed = 0
print len(fastq_files)

okay = 0
non = 0
running = 0
for a_fastq in fastq_files:
    file_id = a_fastq['accession']
    report = get_wfr_out(file_id, 'fastqc-0-11-4-1', None, env)

    if report == 'running':
        running += 1
        print(file_id, 'still running')
        continue

    if report.startswith('no') or not report:
        non += 1
        if run_fastqc == 'fq':
            print 'fastqc running for', a_fastq['accession']
            inp_f = {'input_fastq':file_id}
            run_missing_wfr(step_settings(0), inp_f, file_id, env, tibanna, tag = '')
            time.sleep(30)
        else:
            print 'fastqc run missing for',file_id
    else:
        okay += 1

        
print("okay, not okay, running")    
print(okay, non, running)


Do you wanna run fastqc if missing? (fq/none)
0
okay, not okay, running
(0, 0, 0)


In [20]:
# Release fastq qc

fastq_url = '/search/?status=released&status=released%20to%20project&status=archived&type=FileFastq&limit=all'
fastq_files = ff_utils.search_metadata(fastq_url , ff_env=env)
fastq_files = [i for i in fastq_files if i.get('quality_metric')]
print len(fastq_files)
success = 0
patch_problem = 0

patch_trigger = False
counter1=0
for fastq in fastq_files:
    counter1 += 1
    print counter1
    
    if counter1%100 == 0:
        print counter1
        
    fastq_status = fastq['status']
    if fastq_status in ['released', 'released to project', 'archived']:
        qc_status = ff_utils.get_metadata(fastq['quality_metric']['uuid'],ff_env=env)['status']
        if qc_status != fastq_status:
            print('unmatched status', fastq['accession'])
            patch_data = {"status": fastq_status}
            
            if patch_trigger:
                try:
                    ff_utils.patch_metadata(patch_data, obj_id=fastq['quality_metric'] ,ff_env=env)
                    success += 1
                    print success
                except Exception as e:
                    print e
                    print
                    patch_problem += 1
print success
print patch_problem



KeyboardInterrupt: 